In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lasagne
import theano
import theano.tensor as T

%matplotlib inline

In [2]:
# Read the training data for the neural network
# Input data is 250000x4 and output data is 250000x32
x = pd.read_csv('../parameters_250000.txt', sep=' ')
y = pd.read_csv('../results.txt', sep=' ', index_col=False)

In [40]:
# Make the training data into theano types
X = x.values.astype(theano.config.floatX)
X = X[:1000,:]
Y = y.values.astype(np.float32)
Y = Y[:1000,:]

#### Create the network

In [45]:
# How do I know how large to make the training set?
l_in = lasagne.layers.InputLayer((1000, 4))
# How many hidden layer nodes?  When not specified, weight and bias will be
# initialized automatically (but can customize if I want, and should probably
# be explicit about them eventually to describe my method).
l_hidden = lasagne.layers.DenseLayer(l_in, num_units=50,
                                     nonlinearity=lasagne.nonlinearities
                                     .sigmoid)
# What kind of nonlinearity?
l_out = lasagne.layers.DenseLayer(l_hidden, num_units=32,
                                  nonlinearity=lasagne.nonlinearities.linear)
net_output = lasagne.layers.get_output(l_out)

<b> Question: </b>How do I make the final output layer have a value that is not between 0-1 to compare with the real-valued training data?

#### Objective Function

In [46]:
# As a loss function, we'll use Lasagne's categorical_crossentropy function.
# This allows for the network output to be class probabilities,
# but the target output to be integers denoting the class.
true_output = T.fvector('true_output')
loss = T.mean(lasagne.objectives.squared_error(net_output,
                                                          true_output))

#### Updating the values of weights and biases

In [47]:
# Retrieving all parameters of the network is done using get_all_params,
# which recursively collects the parameters of all layers connected to the
# provided layer.
all_params = lasagne.layers.get_all_params(l_out)
# Now, we'll generate updates using Lasagne's SGD function
updates = lasagne.updates.sgd(loss, all_params, learning_rate=1)
# compile Theano functions for training and computing theoutput.
# Note that because loss depends on the input variable of our input layer,
# we need to retrieve it and tell Theano to use it.
train = theano.function([l_in.input_var, true_output], loss, updates=updates)
get_output = theano.function([l_in.input_var], net_output)

In [48]:
# Train for 100 epochs
for n in xrange(100):
    train(X, Y)

TypeError: ('Bad input argument to theano function with name "<ipython-input-47-26eb679df261>:10"  at index 1(0-based)', 'Wrong number of dimensions: expected 1, got 2 with shape (1000, 32).')

In [ ]:
# Compute the predicted label of the training data.
# The argmax converts the class probability output to class label
y_predicted = np.argmax(get_output(X), axis=1)
# Plot incorrectly classified points as black dots
plt.scatter(X[:, 0], X[:, 1], c=(y != y_predicted), cmap=plt.cm.gray_r)
# Compute and display the accuracy
plt.title("Accuracy: {}%".format(100*np.mean(y == y_predicted)))